
### Cory Melendez
### Assignment: Linear Regression
### CS5262-50

### group stuff together
randn method

In [1813]:
import pandas as pd
from typing import Callable

In [1814]:
food_truck_txt = 'food_truck_data.txt'
housing_txt = 'housing_price_data.txt'

In [1815]:
def get_food_truck_data() -> pd.DataFrame:
    df = pd.read_csv(food_truck_txt, header=None)
    df.columns = ['population', 'profit']

    return df

In [1816]:
food_truck_df = get_food_truck_data()
food_truck_df

,population,profit
0,6.1101,17.59200
1,5.5277,9.13020
2,8.5186,13.66200
3,7.0032,11.85400
4,5.8598,6.82330
...,...,...
92,5.8707,7.20290
93,5.3054,1.98690
94,8.2934,0.14454
95,13.3940,9.05510


## Define a linear model to the data

#### Define our prediction function as

$$
h_\theta(X) = \begin{pmatrix} 1 \\ x_1 \\ ... \\ x_n \end{pmatrix} \cdot \begin{pmatrix} \theta_0 \\ \theta_1 \\ ... \\ \theta_n \end{pmatrix}
$$

$$
= X \cdot \theta
$$

$$
= \theta_0 + \theta_1 x_1 + ... + \theta_n x_n
$$


#### We can then calculate our cost function as the following using least squares:

$$
J(\theta) = (1/2m) \sum_{i=1}^m(h_\theta(X_i) - y_i)^2
$$

#### s.t. $X_i$ is the vector of variables of a row $i$

#### We can then define our gradient descent as:

$$
\theta_j = \theta_j - \alpha (\partial/\partial \theta_j) J(\theta)
$$

####

$$
(\partial/\partial \theta_j) J(\theta) = (\partial/\partial \theta_j) (1/2m) \sum_{i=1}^m(h_\theta(X_i) - y_i)^2
$$

$$
=(1/2m)((\partial/\partial \theta_0)(h_\theta(X_0)-y_0)^2 +
$$

$$
(\partial/\partial \theta_1)(h_\theta(X_1)-y_1)^2 + ... +
$$

$$
(\partial/\partial \theta_m)(h_\theta(X_m)-y_m)^2)
$$

#### Find differential and dont forget chain rule

$$
=(1/2m)(2 (h_\theta(X_0)-y_0) (\partial/\partial \theta_0)(h_\theta(X_0)-y_0) +
$$

$$
2 (h_\theta(X_1)-y_1)(\partial/\partial \theta_1)(h_\theta(X_1)-y_1) + ... +
$$

$$
2 (h_\theta(X_m)-y_m) (\partial/\partial \theta_m)(h_\theta(X_m)-y_m))
$$


#### Remember that:

$$
h_\theta(X) = \begin{pmatrix} 1 \\ x_1 \\ ... \\ x_n \end{pmatrix} \cdot \begin{pmatrix} \theta_0 \\ \theta_1 \\ ... \\ \theta_n \end{pmatrix} = \theta_0 + \theta_1 x_1 + ... + \theta_n x_n
$$

#### So,

$$
= (1/2m)(2 (h_\theta(X_0)-y_0) (\partial/\partial \theta_0)(\theta_0 + \theta_1 x_1 + ... + \theta_n x_n - y_0) +
$$

$$
2 (h_\theta(X_0)- y_1)(\partial/\partial \theta_1)(\theta_0 + \theta_1 x_1 + ... + \theta_n x_n - y_1) + ... +
$$

$$
2 (h_\theta(X_m)-y_m) (\partial/\partial \theta_m)(\theta_0 + \theta_1 x_1 + ... + \theta_m x_m - y_m))
$$


#### Therefore,
$$
(\partial/\partial \theta)J_\theta= (1/m)( (h_\theta(X_0)-y_0) (\partial/\partial \theta_0)(\theta_0 + \theta_1 x_1 + ... + \theta_n x_n - y_0) +
$$

$$
(h_\theta(X_0)- y_1)(\partial/\partial \theta_1)(\theta_0 + \theta_1 x_1 + ... + \theta_n x_n - y_1) + ... +
$$

$$
(h_\theta(X_m)-y_m) (\partial/\partial \theta_m)(\theta_0 + \theta_1 x_1 + ... + \theta_n x_n - y_m))
$$


#### Which can be simplified

$$
(\partial/\partial \theta)J_\theta = (1/m)((h_\theta(X_0) - y_0)(1) +
$$

$$
(h_\theta(X_1) - y_0)(x_1) + ... +
$$

$$
(h_\theta(X_m) - y_m)(x_n))
$$


#### Thus,

$$
(\partial/\partial \theta)J_\theta = (1/m) \sum_{i=0}^m (h_\theta(X_i) - y_i)x_{i,j}
$$



#### So our gradient descent is:
$$
\theta_j := \theta_j - (\alpha/m) \sum_{i=0}^m(h_\theta(X_i) - y_i)x_{i,j}
$$

In [1817]:
def h_theta(x: pd.Series, theta: pd.Series) -> float:
    if theta.count() != x.count():
        raise ValueError(f'x.count: {x.count()} theta.count: {theta.count()}')

    dot_prod = theta[0]
    for i in range(1, x.count()):
        dot_prod += x[i] * theta[i]

    return dot_prod

def j_pde(j: int, x: pd.DataFrame, theta: pd.Series, y: pd.Series, m: int) -> float:
    s:float = 0.0

    #m is number of columns (not including x0)
    #m is the number of data points

    for i in range(1, m+1):
        yi: float = float(y[i-1])
        htheta: float = h_theta(x.iloc[i-1], theta)
        xij: float = float(x.iloc[i-1,j])
        add:float = (yi - htheta) * xij
        print(f'theta={j}, m={i}:({yi} - {htheta}) * {xij} = {add}')
        s += add


    return s

def theta_j(prev_theta_j: float, j: int, alpha: float, x:pd.DataFrame, theta: pd.Series, y: pd.Series) -> float:
    m = x.shape[0]
    jpde = j_pde(j,x,theta, y, m)
    z = prev_theta_j + ((alpha/m) * jpde)
    print(f'theta_{j}= {prev_theta_j} + (({alpha}/{m}) * {jpde}) = {z}')
    return z


In [1818]:
a: float = .1
thetas: pd.Series = pd.Series([2.0,-1.0])
test_df: pd.DataFrame = pd.DataFrame(data={'living_area': [2104, 1600, 2400, 1416, 3000], 'price': [400,330,369,232,540]})
y_actual: pd.Series = test_df['price']
living_area = test_df['living_area']
X = pd.DataFrame(data={0: pd.Series(1, index=range(test_df['living_area'].count())), 1: test_df['living_area']})

In [1819]:
for count in range(0, 1):
    print()
    print(f'theta_0: {thetas[0]}, theta_1: {thetas[1]}\n')
    theta_0 = theta_j(thetas[0], 0, a, X, thetas, y_actual)
    theta_1 = theta_j(thetas[1], 1, a, X, thetas, y_actual)

    thetas[0] = theta_0
    thetas[1] = theta_1







theta_0: 2.0, theta_1: -1.0

theta=0, m=1:(400.0 - -2102.0) * 1.0 = 2502.0
theta=0, m=2:(330.0 - -1598.0) * 1.0 = 1928.0
theta=0, m=3:(369.0 - -2398.0) * 1.0 = 2767.0
theta=0, m=4:(232.0 - -1414.0) * 1.0 = 1646.0
theta=0, m=5:(540.0 - -2998.0) * 1.0 = 3538.0
theta_0= 2.0 + ((0.1/1) * 12381.0) = 1240.1000000000001
theta=1, m=1:(400.0 - -2102.0) * 2104.0 = 5264208.0
theta=1, m=2:(330.0 - -1598.0) * 1600.0 = 3084800.0
theta=1, m=3:(369.0 - -2398.0) * 2400.0 = 6640800.0
theta=1, m=4:(232.0 - -1414.0) * 1416.0 = 2330736.0
theta=1, m=5:(540.0 - -2998.0) * 3000.0 = 10614000.0
theta_1= -1.0 + ((0.1/1) * 27934544.0) = 2793453.4000000004
